In [5]:
import random
import eval7

class Player:
    def __init__(self, name, money):
        self.name = name
        self.money = money
        self.cards = []
        self.current_bet = 0
        self.total_bet = 0

    def bet(self, amount):
        if amount > self.money:
            raise ValueError(f"{self.name} does not have enough money to bet {amount}")
        self.money -= amount
        self.current_bet += amount
        self.total_bet += amount
        return amount

    def reset_bet(self):
        self.current_bet = 0

class PokerGame:
    def __init__(self, players, starting_money=1000):
        self.players = [Player(name, starting_money) for name in players]
        self.pot = 0
        self.deck = self.create_deck()
        self.current_bet = 0
        self.board = []

    def create_deck(self):
        suits = ['h', 'd', 'c', 's']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
        return [rank + suit for suit in suits for rank in ranks]

    def shuffle_deck(self):
        random.shuffle(self.deck)

    def deal_cards(self, num_cards=2):
        for player in self.players:
            player.cards = [self.deck.pop() for _ in range(num_cards)]

    def deal_community_cards(self, num_cards):
        self.board.extend([self.deck.pop() for _ in range(num_cards)])

    def calculate_win_probability(self, player):
        if not player.cards or len(self.board) < 3:
            return random.uniform(0, 1)  # Placeholder for pre-flop estimation

        hole_cards = [eval7.Card(card) for card in player.cards]
        board_cards = [eval7.Card(card) for card in self.board]
        remaining_deck = [eval7.Card(card) for card in self.create_deck() if card not in player.cards and card not in self.board]

        wins, trials = 0, 10000
        for _ in range(trials):
            random.shuffle(remaining_deck)
            remaining_board = board_cards + remaining_deck[:5 - len(board_cards)]
            opp_hole = [remaining_deck[5 - len(board_cards)], remaining_deck[6 - len(board_cards)]]

            player_hand = hole_cards + remaining_board
            opp_hand = opp_hole + remaining_board

            if eval7.evaluate(player_hand) > eval7.evaluate(opp_hand):
                wins += 1

        win_prob = wins / trials

        for other_player in self.players:
            if other_player == player:
                continue
            raise_ratio = other_player.current_bet / other_player.money if other_player.money > 0 else 0

            if raise_ratio > 0.2:
                win_prob -= 0.15
            elif 0.05 <= raise_ratio <= 0.15:
                win_prob -= 0.075
            else:
                win_prob += 0.05

        return max(0, min(1, win_prob))

    def take_bet(self, player, amount):
        bet_amount = player.bet(amount)
        self.pot += bet_amount
        if bet_amount > self.current_bet:
            self.current_bet = bet_amount

    def betting_round(self):
        num_players = len(self.players)
        threshold_high = 100 * (num_players - 1) / num_players / 100

        for player in self.players:
            win_prob = self.calculate_win_probability(player)
            print(f"{player.name}'s turn. Adjusted Win Probability: {win_prob:.2f}, Current bet to match: {self.current_bet}")

            if win_prob > threshold_high:
                bet_amount = int(0.2 * self.pot)
                print(f"{player.name} raises by 20% of the pot ({bet_amount})")
            elif 0.6 <= win_prob <= threshold_high:
                bet_amount = int(0.05 * self.pot)
                print(f"{player.name} raises by 5% of the pot ({bet_amount})")
            elif 0.4 <= win_prob < 0.6:
                bet_amount = self.current_bet
                print(f"{player.name} calls the bet ({bet_amount})")
            else:
                print(f"{player.name} folds.")
                continue

            self.take_bet(player, bet_amount)
            print(f"{player.name} bets {bet_amount}. Remaining money: {player.money}")

        for player in self.players:
            player.reset_bet()

    def show_player_status(self):
        for player in self.players:
            print(f"{player.name} - Money: {player.money}, Cards: {player.cards}, Total Bet: {player.total_bet}")

    def play(self):
        self.shuffle_deck()
        self.deal_cards()
        self.deal_community_cards(3)  # Flop
        print("Starting betting round...")
        self.betting_round()
        self.deal_community_cards(1)  # Turn
        self.betting_round()
        self.deal_community_cards(1)  # River
        self.betting_round()
        print("Final pot size:", self.pot)
        self.show_player_status()

if __name__ == "__main__":
    players = ["Alice", "Bob", "Charlie", "David"]
    game = PokerGame(players)
    game.play()

Starting betting round...
Alice's turn. Adjusted Win Probability: 0.42, Current bet to match: 0
Alice calls the bet (0)
Alice bets 0. Remaining money: 1000
Bob's turn. Adjusted Win Probability: 0.72, Current bet to match: 0
Bob raises by 5% of the pot (0)
Bob bets 0. Remaining money: 1000
Charlie's turn. Adjusted Win Probability: 0.45, Current bet to match: 0
Charlie calls the bet (0)
Charlie bets 0. Remaining money: 1000
David's turn. Adjusted Win Probability: 0.48, Current bet to match: 0
David calls the bet (0)
David bets 0. Remaining money: 1000
Alice's turn. Adjusted Win Probability: 0.32, Current bet to match: 0
Alice folds.
Bob's turn. Adjusted Win Probability: 0.68, Current bet to match: 0
Bob raises by 5% of the pot (0)
Bob bets 0. Remaining money: 1000
Charlie's turn. Adjusted Win Probability: 0.44, Current bet to match: 0
Charlie calls the bet (0)
Charlie bets 0. Remaining money: 1000
David's turn. Adjusted Win Probability: 0.38, Current bet to match: 0
David folds.
Alice's 

In [3]:
!pip install eval7
import eval7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 12.2 MB/s eta 0:00:00
